# Introduction to Kubernetes

**Learning Objectives**
 * Create GKE cluster from command line
 * Deploy an application to your cluster
 * Cleanup, delete the cluster 

## Overview
Kubernetes is an open source project (available on [kubernetes.io](kubernetes.io)) which can run on many different environments, from laptops to high-availability multi-node clusters; from public clouds to on-premise deployments; from virtual machines to bare metal.

The goal of this lab is to provide a short introduction to Kubernetes (k8s) and some basic functionality.

## Setup

To run this lab, you may need to run the following command:

```bash
../../../scripts/setup_kubernetes_auth.sh
```

## Create a GKE cluster

A cluster consists of at least one cluster master machine and multiple worker machines called nodes. Nodes are Compute Engine virtual machine (VM) instances that run the Kubernetes processes necessary to make them part of the cluster.

**Note**: Cluster names must start with a letter and end with an alphanumeric, and cannot be longer than 40 characters.

We'll call our cluster `asl-cluster`.

In [ ]:
import os

CLUSTER_NAME = "asl-cluster"
ZONE = "us-central1-a"

os.environ["CLUSTER_NAME"] = CLUSTER_NAME
os.environ["ZONE"] = ZONE

We'll set our default compute zone to `us-central1-a` and use `gcloud container clusters create ...` to create the GKE cluster. Let's first look at all the clusters we currently have. 

In [ ]:
!gcloud container clusters list

Then we'll use `gcloud container clusters create` to create a new cluster using the `CLUSTER_NAME` we set above. This takes a few minutes...

In [ ]:
%%bash
gcloud config set compute/zone ${ZONE}
gcloud container clusters create ${CLUSTER_NAME}

Now when we list our clusters again, we should see the cluster we created. 

In [ ]:
!gcloud container clusters list

## Get authentication credentials and deploy and application

After creating your cluster, you need authentication credentials to interact with it. Use `get-credentials` to authenticate the cluster.

In [ ]:
%%bash
gcloud container clusters get-credentials ${CLUSTER_NAME}

You can now deploy a containerized application to the cluster. For this lab, you'll run `hello-app` in your cluster.

GKE uses Kubernetes objects to create and manage your cluster's resources. Kubernetes provides the [Deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/) object for deploying stateless applications like web servers. [Service](https://kubernetes.io/docs/concepts/services-networking/service/) objects define rules and load balancing for accessing your application from the internet.

Use the `kubectl create` command to create a new Deployment `hello-server` from the `hello-app` container image. The `--image` flag to specify a container image to deploy. The `kubectl create` command pulls the example image from a Artifact Registry bucket. Here, use [us-docker.pkg.dev/google-samples/containers/gke/hello-app:1.0](https://us-docker.pkg.dev/google-samples/containers/gke/hello-app:1.0) to indicate the specific image version to pull. If a version is not specified, the latest version is used.

In [ ]:
!kubectl create deployment hello-server --image=us-docker.pkg.dev/google-samples/containers/gke/hello-app:1.0

This Kubernetes command creates a Deployment object that represents `hello-server`. To create a Kubernetes Service, which is a Kubernetes resource that lets you expose your application to external traffic, run the `kubectl expose` command. 

In this command, 
  * `--port` specifies the port that the container exposes.
  * `type="LoadBalancer"` creates a Compute Engine load balancer for your container.

In [ ]:
!kubectl expose deployment hello-server --type=LoadBalancer --port 8080

Use the `kubectl get service` command to inspect the `hello-server` Service.

**Note**: It might take a minute for an external IP address to be generated. Run the previous command again if the `EXTERNAL-IP` column for `hello-server` status is pending.

In [ ]:
!kubectl get service

You can now view the application from your web browser, open a new tab and enter the following address, replacing `EXTERNAL IP` with the EXTERNAL-IP for `hello-server`:

```bash
http://[EXTERNAL_IP]:8080
```

You should see a simple page which displays

```bash
Hello, world!
Version: 1.0.0
Hostname: hello-server-5bfd595c65-7jqkn
```

## Cleanup

Delete the cluster using `gcloud` to free up those resources. Use the `--quiet` flag if you are executing this in a notebook. Deleting the cluster can take a few minutes. 

In [ ]:
%%bash
gcloud container clusters --quiet delete ${CLUSTER_NAME}

Copyright 2020 Google LLC Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.